In [2]:
import pandas as pd

In [1]:
directory = 'drive/MyDrive/Data'
cohort_name = 'UKBB'

In [3]:
annovar_path = f'{directory}/{cohort_name}_withgnomad_clinvar.annovar.hg38_multianno.txt'
annovar_df = pd.read_csv(annovar_path, sep='\t')
columns_to_extract = [
    'Chr', 'Start', 'End', 'Ref', 'Alt', 'Func.refGene',
    'GeneDetail.refGene', 'ExonicFunc.refGene', 'AAChange.refGene',
    'CADD_phred', 'CLNSIG', 'gnomad41_genome_AF',
    'gnomad41_genome_AF_asj', 'gnomad41_genome_AF_nfe','avsnp151']

# Extract the relevant columns
annovar_df = annovar_df[columns_to_extract]

# Display the first few rows of the DataFrame
annovar_df.head()

,Chr,Start,End,Ref,Alt,Func.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,CADD_phred,CLNSIG,gnomad41_genome_AF,gnomad41_genome_AF_asj,gnomad41_genome_AF_nfe,avsnp151
0,chr12,40225042,40225042,C,T,UTR5,NM_198578:c.-90C>T,.,.,.,.,.,.,.,.
1,chr12,40225043,40225043,C,G,UTR5,NM_198578:c.-89C>G,.,.,.,.,.,.,.,.
2,chr12,40225044,40225044,-,G,UTR5,NM_198578:c.-88_-87insG,.,.,.,.,.,.,.,.
3,chr12,40225046,40225046,G,A,UTR5,NM_198578:c.-86G>A,.,.,.,.,.,.,.,.
4,chr12,40225047,40225047,G,A,UTR5,NM_198578:c.-85G>A,.,.,.,.,1.314e-05,0,2.94e-05,rs1020433882


In [4]:
# Define the path to the freq and counts files
freq_path = f'{directory}/{cohort_name}_MAF.assoc.fisher'
counts_path = f'{directory}/{cohort_name}.assoc.fisher'
freq_df = pd.read_csv(freq_path, sep='\s+')
counts_df = pd.read_csv(counts_path, sep='\s+')

# Format data types to match
freq_df['SNP'] = freq_df['SNP'].astype(str)
counts_df['SNP'] = counts_df['SNP'].astype(str)

# merge the two files
MAF_and_counts_df = pd.merge(counts_df[['CHR', 'SNP', 'BP', 'A1', 'A2', 'C_A', 'C_U']],
                             freq_df[['SNP', 'F_A', 'F_U']],
                             on='SNP')

# Display the first few rows of the merged DataFrame
MAF_and_counts_df.head()


,CHR,SNP,BP,A1,A2,C_A,C_U,F_A,F_U
0,12,chr12_40225042_C_T,40225042,T,C,0,0,0.0,0.0
1,12,chr12_40225043_C_G,40225043,G,C,0,0,0.0,0.0
2,12,chr12_40225044_C_CG,40225044,CG,C,0,0,0.0,0.0
3,12,chr12_40225046_G_A,40225046,A,G,0,0,0.0,0.0
4,12,chr12_40225047_G_A,40225047,A,G,0,0,0.0,0.0


In [5]:
# Count number of rows
print(f'Number of rows: {len(MAF_and_counts_df)}')

Number of rows: 2801


In [6]:
# Based on CHR, A1 and A2 columns, merge the annovar and MAF_and_counts dataframes
MAF_and_counts_df['CHR'] = MAF_and_counts_df['CHR'].astype(str)
MAF_and_counts_df['A1'] = MAF_and_counts_df['A1'].astype(str)
MAF_and_counts_df['A2'] = MAF_and_counts_df['A2'].astype(str)
annovar_df['Chr'] = annovar_df['Chr'].astype(str)
if cohort_name=='UKBB':
  # remove Chr from 'Chr'
  annovar_df['Chr'] = annovar_df['Chr'].str.replace('chr', '')
annovar_df['Ref'] = annovar_df['Ref'].astype(str)
annovar_df['Alt'] = annovar_df['Alt'].astype(str)
merged_df = pd.merge(annovar_df, MAF_and_counts_df, left_on=['Chr', 'Start', 'Ref', 'Alt'], right_on=['CHR', 'BP', 'A2', 'A1'])
merged_df = merged_df.drop(columns=['CHR', 'BP', 'A1', 'A2'])
# Display the first few rows of the merged DataFrame
merged_df.head()

,Chr,Start,End,Ref,Alt,Func.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,CADD_phred,CLNSIG,gnomad41_genome_AF,gnomad41_genome_AF_asj,gnomad41_genome_AF_nfe,avsnp151,SNP,C_A,C_U,F_A,F_U
0,12,40225042,40225042,C,T,UTR5,NM_198578:c.-90C>T,.,.,.,.,.,.,.,.,chr12_40225042_C_T,0,0,0.0,0.0
1,12,40225043,40225043,C,G,UTR5,NM_198578:c.-89C>G,.,.,.,.,.,.,.,.,chr12_40225043_C_G,0,0,0.0,0.0
2,12,40225046,40225046,G,A,UTR5,NM_198578:c.-86G>A,.,.,.,.,.,.,.,.,chr12_40225046_G_A,0,0,0.0,0.0
3,12,40225047,40225047,G,A,UTR5,NM_198578:c.-85G>A,.,.,.,.,1.314e-05,0,2.94e-05,rs1020433882,chr12_40225047_G_A,0,0,0.0,0.0
4,12,40225049,40225049,G,A,UTR5,NM_198578:c.-83G>A,.,.,.,.,.,.,.,.,chr12_40225049_G_A,0,0,0.0,0.0


In [7]:
# Add domain annotation based on BP location
domain_df = pd.read_excel(f'{directory}/LRRK2_GRCh38.xlsx', sheet_name="Sheet2")

domain_df.head()

,Chromosome,Gene,Domain,Column1,Column12,narrow_start,narrow_end,wide_start,wide_end
0,12,LRRK2,ARM,100-688,100-688,40225234,40274891,40225153,40277914
1,12,LRRK2,ANK,688-863,688-863,40277916,40284040,40277915,40284043
2,12,LRRK2,LRR,943-1309,943-1309,40294854,40305881,40294854,40305881
3,12,LRRK2,RocCOR,1327-1842,1327-1842,40309151,40313983,40308486,40322084
4,12,LRRK2,Kinase,1879-2138,1879-2138,40323303,40340349,40323249,40351571


In [8]:
def annotate_domain(row, domain_df):
    for _, domain_row in domain_df.iterrows():
        if domain_row['wide_start'] <= row['Start'] <= domain_row['wide_end']:
            return domain_row['Domain']
    return 'Unknown'

# Annotate domain information of variants
merged_df['Domain'] = merged_df.apply(annotate_domain, axis=1, domain_df=domain_df)

# Display the first few rows of the merged DataFrame
merged_df.head()
# Count each domain group
domain_counts = merged_df['Domain'].value_counts()
print(domain_counts)

Domain
ARM        759
RocCOR     454
Unknown    368
WD40       365
LRR        355
Kinase     277
ANK        141
Name: count, dtype: int64


In [9]:
# Count number of rows
print(f'Number of rows: {len(merged_df)}')

# Show me the unmatched rows from MAF_and_counts_df
unmatched_rows = MAF_and_counts_df[~MAF_and_counts_df['SNP'].isin(merged_df['SNP'])]
unmatched_rows.head()
# This is before the UTR5, intronic, and UTR3 variants are removed

Number of rows: 2719


,CHR,SNP,BP,A1,A2,C_A,C_U,F_A,F_U
2,12,chr12_40225044_C_CG,40225044,CG,C,0,0,0.0,0.000000
60,12,chr12_40225280_G_A,40225280,G,A,0,3,0.0,0.000024
77,12,chr12_40225338_T_TA,40225338,TA,T,0,0,0.0,0.000000
84,12,chr12_40225529_CTT_C,40225529,C,CTT,0,0,0.0,0.000000
86,12,chr12_40225533_T_TC,40225533,TC,T,0,0,0.0,0.000000


In [11]:
# Remove UTR5, intronic, and UTR3 variants and recount unmatched
merged_df = merged_df[~merged_df['Func.refGene'].isin(['UTR5', 'intronic', 'UTR3'])]
print(f'Number of rows: {len(merged_df)}')

# Count again
unmatched_rows = MAF_and_counts_df[~MAF_and_counts_df['SNP'].isin(merged_df['SNP'])]
unmatched_rows.head()
# This is after the UTR5, intronic, and UTR3 variants are removed
# Count each domain group
domain_counts = merged_df['Domain'].value_counts()
print(domain_counts)

Number of rows: 1519
Domain
ARM        473
RocCOR     285
WD40       254
LRR        245
Kinase     163
ANK         99
Name: count, dtype: int64


In [12]:
# remove Domain == Unknown
merged_df = merged_df[merged_df['Domain'] != 'Unknown']
print(f'Number of rows: {len(merged_df)}')

Number of rows: 1519


In [408]:
# Make the headers more legible
# Mapping of old headers to new headers without underscores
header_mapping = {
    'Chr': 'Chromosome',
    'Start': 'Start Position',
    'End': 'End Position',
    'Ref': 'Reference Allele',
    'Alt': 'Alternate Allele',
    'Func.refGene': 'Functional Annotation',
    'GeneDetail.refGene': 'outside gene cDNA Change',
    'ExonicFunc.refGene': 'Exonic Function',
    'AAChange.refGene': 'cDNA and Amino Acid Change',
    'CADD_phred': 'CADD v1.7 Phred Score',
    'CLNSIG': 'Clinical Significance (Varsome)',
    'gnomad41_genome_AF': 'gnomAD Allele Frequency',
    'gnomad41_genome_AF_asj': 'gnomAD AF Ashkenazi',
    'gnomad41_genome_AF_nfe': 'gnomAD AF Non Finnish European',
    'avsnp151': 'rsID'
}
merged_df = merged_df.rename(columns=header_mapping)

# Display the first few rows of the merged DataFrame
merged_df.head()

,Chromosome,Start Position,End Position,Reference Allele,Alternate Allele,Functional Annotation,outside gene cDNA Change,Exonic Function,cDNA and Amino Acid Change,CADD v1.7 Phred Score,...,gnomAD Allele Frequency,gnomAD AF Ashkenazi,gnomAD AF Non Finnish European,rsID,SNP,C_A,C_U,F_A,F_U,Domain
31,12,40225154,40225154,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G23A:p.G8E,10.65,...,6.571e-06,0,0,rs1052256144,chr12_40225154_G_A,0,0,0.0,0.000000,ARM
32,12,40225159,40225159,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G28A:p.E10K,15.39,...,.,.,.,rs281865040,chr12_40225159_G_A,0,0,0.0,0.000000,ARM
33,12,40225160,40225160,A,G,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.A29G:p.E10G,15.15,...,.,.,.,.,chr12_40225160_A_G,0,0,0.0,0.000000,ARM
34,12,40225164,40225164,G,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G33C:p.E11D,14.00,...,.,.,.,.,chr12_40225164_G_C,0,0,0.0,0.000000,ARM
35,12,40225170,40225170,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon1:c.G39A:p.E13E,.,...,.,.,.,rs775407458,chr12_40225170_G_A,0,2,0.0,0.000016,ARM


In [409]:
header_mapping_2 = {
    'C_A': 'N (Case)',
    'C_U': 'N (Control)',
    'F_A': 'MAF (Case)',
    'F_U': 'MAF (Control)'
}

merged_df = merged_df.rename(columns=header_mapping_2)
merged_df

,Chromosome,Start Position,End Position,Reference Allele,Alternate Allele,Functional Annotation,outside gene cDNA Change,Exonic Function,cDNA and Amino Acid Change,CADD v1.7 Phred Score,...,gnomAD Allele Frequency,gnomAD AF Ashkenazi,gnomAD AF Non Finnish European,rsID,SNP,N (Case),N (Control),MAF (Case),MAF (Control),Domain
31,12,40225154,40225154,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G23A:p.G8E,10.65,...,6.571e-06,0,0,rs1052256144,chr12_40225154_G_A,0,0,0.0,0.000000,ARM
32,12,40225159,40225159,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G28A:p.E10K,15.39,...,.,.,.,rs281865040,chr12_40225159_G_A,0,0,0.0,0.000000,ARM
33,12,40225160,40225160,A,G,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.A29G:p.E10G,15.15,...,.,.,.,.,chr12_40225160_A_G,0,0,0.0,0.000000,ARM
34,12,40225164,40225164,G,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G33C:p.E11D,14.00,...,.,.,.,.,chr12_40225164_G_C,0,0,0.0,0.000000,ARM
35,12,40225170,40225170,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon1:c.G39A:p.E13E,.,...,.,.,.,rs775407458,chr12_40225170_G_A,0,2,0.0,0.000016,ARM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,12,40367733,40367733,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon51:c.G7552A:p.A2518T,18.07,...,.,.,.,rs755075477,chr12_40367733_G_A,0,4,0.0,0.000032,WD40
2693,12,40367734,40367734,C,T,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon51:c.C7553T:p.A2518V,21.6,...,.,.,.,.,chr12_40367734_C_T,0,0,0.0,0.000000,WD40
2694,12,40367736,40367736,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon51:c.G7555A:p.E2519K,16.38,...,.,.,.,.,chr12_40367736_G_A,0,0,0.0,0.000000,WD40
2695,12,40367737,40367737,A,G,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon51:c.A7556G:p.E2519G,14.60,...,.,.,.,.,chr12_40367737_A_G,0,0,0.0,0.000000,WD40


In [410]:
# Now we organize the cDNA and Amino Acid Change column and separate into cDNA and AA Change separate columns
for index, row in merged_df.iterrows():
    if pd.notna(row['cDNA and Amino Acid Change']):
        split_values = row['cDNA and Amino Acid Change'].split(':')[-2:]
        merged_df.at[index, 'cDNA'] = split_values[0]
        merged_df.at[index, 'Amino Acid Change'] = split_values[1] if len(split_values) > 1 else None

merged_df

,Chromosome,Start Position,End Position,Reference Allele,Alternate Allele,Functional Annotation,outside gene cDNA Change,Exonic Function,cDNA and Amino Acid Change,CADD v1.7 Phred Score,...,gnomAD AF Non Finnish European,rsID,SNP,N (Case),N (Control),MAF (Case),MAF (Control),Domain,cDNA,Amino Acid Change
31,12,40225154,40225154,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G23A:p.G8E,10.65,...,0,rs1052256144,chr12_40225154_G_A,0,0,0.0,0.000000,ARM,c.G23A,p.G8E
32,12,40225159,40225159,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G28A:p.E10K,15.39,...,.,rs281865040,chr12_40225159_G_A,0,0,0.0,0.000000,ARM,c.G28A,p.E10K
33,12,40225160,40225160,A,G,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.A29G:p.E10G,15.15,...,.,.,chr12_40225160_A_G,0,0,0.0,0.000000,ARM,c.A29G,p.E10G
34,12,40225164,40225164,G,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G33C:p.E11D,14.00,...,.,.,chr12_40225164_G_C,0,0,0.0,0.000000,ARM,c.G33C,p.E11D
35,12,40225170,40225170,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon1:c.G39A:p.E13E,.,...,.,rs775407458,chr12_40225170_G_A,0,2,0.0,0.000016,ARM,c.G39A,p.E13E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,12,40367733,40367733,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon51:c.G7552A:p.A2518T,18.07,...,.,rs755075477,chr12_40367733_G_A,0,4,0.0,0.000032,WD40,c.G7552A,p.A2518T
2693,12,40367734,40367734,C,T,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon51:c.C7553T:p.A2518V,21.6,...,.,.,chr12_40367734_C_T,0,0,0.0,0.000000,WD40,c.C7553T,p.A2518V
2694,12,40367736,40367736,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon51:c.G7555A:p.E2519K,16.38,...,.,.,chr12_40367736_G_A,0,0,0.0,0.000000,WD40,c.G7555A,p.E2519K
2695,12,40367737,40367737,A,G,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon51:c.A7556G:p.E2519G,14.60,...,.,.,chr12_40367737_A_G,0,0,0.0,0.000000,WD40,c.A7556G,p.E2519G


In [411]:
# Remove variants where both N (Case) and N (Control) are 0

#Count the excluded
excluded_df = merged_df[(merged_df['N (Case)'] == 0) & (merged_df['N (Control)'] == 0)]
print(f'Number of excluded rows: {len(excluded_df)}')


merged_df = merged_df[~((merged_df['N (Case)'] == 0) & (merged_df['N (Control)'] == 0))]


Number of excluded rows: 1065


In [412]:
# print column names
print(merged_df.columns)

Index(['Chromosome', 'Start Position', 'End Position', 'Reference Allele',
       'Alternate Allele', 'Functional Annotation', 'outside gene cDNA Change',
       'Exonic Function', 'cDNA and Amino Acid Change',
       'CADD v1.7 Phred Score', 'Clinical Significance (Varsome)',
       'gnomAD Allele Frequency', 'gnomAD AF Ashkenazi',
       'gnomAD AF Non Finnish European', 'rsID', 'SNP', 'N (Case)',
       'N (Control)', 'MAF (Case)', 'MAF (Control)', 'Domain', 'cDNA',
       'Amino Acid Change'],
      dtype='object')


In [413]:
# if the "outside gene cDNA Change" column is not empty then split by ":" then pick third element and store it inside the cDNA of that row
for index, row in merged_df.iterrows():
    if pd.notna(row['outside gene cDNA Change']):
        split_values = row['outside gene cDNA Change'].split(':')
        if len(split_values) >= 3:
            merged_df.at[index, 'cDNA'] = split_values[2]
            # print cDNA at index
            print(merged_df.at[index, 'cDNA'])

c.151+1G>A
c.151+2T>C
c.838+1G>A
c.1102-1G>A
c.1656+1G>A
c.5015+2T>C
c.7028+1G>A


In [414]:
# reorder columns such that
# Domain	Chromosome	Position	Reference Allele	Alternate Allele	cDNA	Amino Acid Change	Functional Annotation	Exonic Function	N (Case)	N (Control)	MAF (Case)	MAF (Control)	CADD Phred Score	Clinical Significance (Varsome)	gnomAD Allele Frequency	gnomAD AF Ashkenazi	gnomAD AF Non Finnish European
ordered_merged_df = merged_df[[
    'Domain', 'Chromosome', 'Start Position', 'Reference Allele',
    'Alternate Allele', 'cDNA', 'Amino Acid Change','rsID', 'Functional Annotation',
    'Exonic Function', 'N (Case)', 'N (Control)', 'MAF (Case)',
    'MAF (Control)', 'CADD v1.7 Phred Score', 'Clinical Significance (Varsome)',
    'gnomAD Allele Frequency', 'gnomAD AF Ashkenazi',
    'gnomAD AF Non Finnish European'
]]

ordered_merged_df.head()

,Domain,Chromosome,Start Position,Reference Allele,Alternate Allele,cDNA,Amino Acid Change,rsID,Functional Annotation,Exonic Function,N (Case),N (Control),MAF (Case),MAF (Control),CADD v1.7 Phred Score,Clinical Significance (Varsome),gnomAD Allele Frequency,gnomAD AF Ashkenazi,gnomAD AF Non Finnish European
35,ARM,12,40225170,G,A,c.G39A,p.E13E,rs775407458,exonic,synonymous SNV,0,2,0.000000,0.000016,.,Likely_benign,.,.,.
36,ARM,12,40225176,T,C,c.T45C,p.T15T,rs142399623,exonic,synonymous SNV,9,67,0.001578,0.000536,.,Likely_benign,0.0002,0,0.0004
40,ARM,12,40225188,G,A,c.G57A,p.L19L,.,exonic,synonymous SNV,0,2,0.000000,0.000016,.,.,.,.,.
50,ARM,12,40225260,G,C,c.G129C,p.L43L,rs550441220,exonic,synonymous SNV,0,1,0.000000,0.000008,.,Likely_benign,1.314e-05,0,2.94e-05
53,ARM,12,40225268,C,T,c.C137T,p.T46M,rs781394575,exonic,nonsynonymous SNV,0,5,0.000000,0.000040,16.95,Uncertain_significance,3.286e-05,0,5.881e-05


In [415]:
# save into excel file
ordered_merged_df.to_excel(f'{directory}/{cohort_name}_annotated.xlsx', index=False)
